# Проверка загрузки MS_ResUNet из файла (.pt)

Этот ноутбук делает только одно: **проверяет, что ваша модель корректно создаётся и загружается из файла** (state_dict / checkpoint) и показывает `missing/unexpected` ключи.

In [1]:
from pathlib import Path
import torch

# --- НАСТРОЙКИ ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# УКАЖИТЕ ПУТЬ К ВАШЕМУ .pt (state_dict или checkpoint)
MODEL_PT = Path(r"C:\Users\Вячеслав\Documents\superresolution\models\best_x2_ms_resunet.pt")  # <-- поменяйте на ваш путь

# Если хотите строгое совпадение ключей, поставьте True.
# На этапе диагностики удобнее False, чтобы увидеть missing/unexpected.
STRICT = False

print("DEVICE:", DEVICE)
print("MODEL_PT exists:", MODEL_PT.exists(), "->", MODEL_PT)

DEVICE: cuda
MODEL_PT exists: True -> C:\Users\Вячеслав\Documents\superresolution\models\best_x2_ms_resunet.pt


In [2]:
def _strip_prefix(state_dict: dict, prefix: str = "module.") -> dict:
    """Убирает префикс 'module.' (часто появляется после DataParallel/DDP)."""
    if not isinstance(state_dict, dict):
        raise TypeError("state_dict must be a dict")
    if not any(k.startswith(prefix) for k in state_dict.keys()):
        return state_dict
    return {k[len(prefix):] if k.startswith(prefix) else k: v for k, v in state_dict.items()}

def extract_state_dict(ckpt) -> dict:
    """Достаёт state_dict из разных форматов сохранения."""
    # 1) чистый state_dict
    if isinstance(ckpt, dict) and all(isinstance(k, str) for k in ckpt.keys()):
        # часто checkpoint хранит state_dict под одним из этих ключей
        for key in ("state_dict", "model", "model_state_dict", "net", "generator"):
            if key in ckpt and isinstance(ckpt[key], dict):
                sd = ckpt[key]
                return _strip_prefix(sd)
        # если похоже на state_dict (ключи вида 'conv.weight' и т.п.)
        # (эвристика: хотя бы один ключ содержит '.weight' или '.bias')
        if any((".weight" in k) or (".bias" in k) for k in ckpt.keys()):
            return _strip_prefix(ckpt)

    raise ValueError(
        "Не удалось извлечь state_dict. "
        "Ожидается либо state_dict напрямую, либо checkpoint с ключом вроде "
        "'model' / 'state_dict' / 'model_state_dict'."
    )

def load_checkpoint(path: Path, map_location="cpu"):
    """Безопасная загрузка torch checkpoint/state_dict."""
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(path)
    return torch.load(path, map_location=map_location)

## Импорт вашей архитектуры

Ниже **поменяйте импорт** на ваш файл/модуль, где определена функция `MS_ResUNet()` (или класс).

Пример:
- если у вас `models/ms_resunet.py` и внутри `def MS_ResUNet(): ...`
  ```python
  from models.ms_resunet import MS_ResUNet
  ```


In [5]:
# для использования ячейки ниже нужно перейти в папку с модулем, менять, если путь изменился
%cd C:\Users\Вячеслав\Documents\superresolution\modules

C:\Users\Вячеслав\Documents\superresolution\modules


In [7]:
# функция-конструктор
from ms_resunet import MS_ResUNet

assert MS_ResUNet is not None, "Поменяйте импорт выше так, чтобы MS_ResUNet был доступен."

In [8]:
# 1) Загружаем checkpoint/state_dict
ckpt = load_checkpoint(MODEL_PT, map_location="cpu")
state = extract_state_dict(ckpt)

print("Loaded keys:", len(state))
first_keys = list(state.keys())[:10]
print("First keys sample:")
for k in first_keys:
    print("  ", k)

Loaded keys: 363
First keys sample:
   conv1.weight
   bn1.weight
   bn1.bias
   bn1.running_mean
   bn1.running_var
   bn1.num_batches_tracked
   upCT4.weight
   upCT4.bias
   upCT3.weight
   upCT3.bias


In [9]:
# 2) Создаём модель и загружаем веса
model = MS_ResUNet()
model.to(DEVICE)

missing, unexpected = model.load_state_dict(state, strict=STRICT)
model.eval()

# 3) Выводим диагностику
print("\n=== load_state_dict report ===")
print("STRICT:", STRICT)
print("missing keys:", len(missing))
print("unexpected keys:", len(unexpected))

if missing:
    print("\n[missing] sample (up to 30):")
    for k in missing[:30]:
        print("  ", k)

if unexpected:
    print("\n[unexpected] sample (up to 30):")
    for k in unexpected[:30]:
        print("  ", k)

# 4) Кол-во параметров
n_params = sum(p.numel() for p in model.parameters())
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nparams total: {n_params:,} | trainable: {n_trainable:,}")


=== load_state_dict report ===
STRICT: False
missing keys: 0
unexpected keys: 0

params total: 26,687,841 | trainable: 26,687,841


## (Опционально) Быстрый forward-test на одном изображении

Если у вас модель SR `x2`, и вход — **grayscale** `[1, 1, H, W]` в диапазоне `[0,1]`,
то следующая ячейка:
- читает картинку,
- прогоняет через модель,
- сохраняет результат `sr_debug.png`.

Если у вас другой формат входа (RGB/YCbCr/16-bit), просто адаптируйте предобработку.


In [10]:
from PIL import Image
import numpy as np

# Укажите путь к тестовому LR изображению (обычно png/jpg)
LR_IMG = Path(r"C:\Users\Вячеслав\Documents\superresolution\DeepRockSR-2D\carbonate2D\carbonate2D_test_LR_default_X2\3607x2.png")  # <-- поменяйте

OUT_IMG = Path("sr_debug.png")

assert LR_IMG.exists(), f"LR_IMG не найден: {LR_IMG}"

# читаем как grayscale float32 [0,1]
img = Image.open(LR_IMG).convert("L")
arr = np.array(img, dtype=np.float32) / 255.0  # [H,W]
x = torch.from_numpy(arr)[None, None, ...].to(DEVICE)  # [1,1,H,W]

with torch.no_grad():
    y = model(x)

# если модель возвращает список/кортеж — берём первый элемент
if isinstance(y, (list, tuple)):
    y = y[0]

y = y.detach().float().clamp(0, 1)[0, 0].cpu().numpy()
y8 = (y * 255.0 + 0.5).astype(np.uint8)
Image.fromarray(y8, mode="L").save(OUT_IMG)

print("Saved:", OUT_IMG.resolve())

Saved: C:\Users\Вячеслав\Documents\superresolution\modules\sr_debug.png


C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_11892\4096709630.py:25: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(y8, mode="L").save(OUT_IMG)
